In [17]:
from dotenv import load_dotenv
import os


load_dotenv(dotenv_path=".env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

In [18]:
from pydantic import BaseModel, Field


class Persona(BaseModel):
    name: str = Field(..., description="ペルソナの名前")
    background: str = Field(..., description="ペルソナの持つ背景情報")

class Personas(BaseModel):
    personas: list[Persona] = Field(default_factory=list, description="ペルソナのリスト")

class Interview(BaseModel):
    persona: Persona = Field(..., description="インタビュー対象のペルソナ")
    question: str = Field(..., description="インタビューの質問")
    answer: str = Field(..., description="インタビューの回答")

class InterviewResult(BaseModel):
    interviews: list[Interview] = Field(default_factory=list, description="インタビュー結果のリスト")

class EvaluationResult(BaseModel):
    reason: str = Field(..., description="判断の理由")
    is_sufficient: bool = Field(..., description="情報が十分かどうか")

In [19]:
from typing import Annotated
import operator

class InterviewState(BaseModel):
    user_request: str = Field(..., description="ユーザーからのリクエスト")
    personas: Annotated[list[Persona], operator.add] = Field(default_factory=list, description="生成されたペルソナのリスト")
    interviews: Annotated[list[Interview], operator.add] = Field(default_factory=list, description="実施されたインタビューのリスト")
    requirements_doc: str = Field("", description="生成された要件定義書")
    iteration: int = Field(0, description="ペルソナ生成とインタビューの反復回数")
    is_information_sufficient: bool = Field(False, description="情報が十分かどうか")

In [20]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

class PersonaGenerator:
    def __init__(self, llm: ChatOpenAI, k: int = 5):
        self.llm = llm.with_structured_output(Personas)
        self.k = k
    
    def run(self, user_request: str) -> list[Persona]:
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたはユーザーインタビュー用の多様なペルソナを作成する専門家です。"
                ),
                (
                    "user",
                    f"以下のユーザーリクエストに関するインタビュー用に、{self.k}人の多様なペルソナを作成してください。\n\n"
                    "ユーザーリクエスト: {user_request}\n\n"
                    "各ペルソナには名前と簡単な背景情報を含む必要があります。年齢、性別、職業、技術的専門知識において多様性を確保してください。"
                ),
            ]
        )
        chain = prompt | self.llm
        return chain.invoke({"user_request": user_request})

In [21]:
from langchain_core.output_parsers import StrOutputParser

class InterviewConductor:
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm

    def run(self, user_request: str, personas: list[Persona]) -> list[Interview]:
        questions = self._generate_questions(user_request=user_request, personas=personas)
        answers = self._generate_answers(personas=personas, questions=questions)
        interviews = self._create_interviews(personas=personas, questions=questions, answers=answers)
        return InterviewResult(interviews=interviews)
    
    def _generate_questions(self, user_request: str, personas: list[Persona]) -> list[str]:
        question_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたはユーザー要件に基づいて適切な質問を生成する専門家です。"
                ),
                (
                    "user",
                    f"以下のペルソナに関連するユーザーリクエストに基づいて、1つの質問を生成してください。\n\n"
                    "ユーザーリクエスト: {user_request}\n"
                    "ペルソナ:{persona_name} - {persona_background}\n\n"
                    "質問は具体的で、このペルソナの視点から重要な情報を引き出すように設計してください。"
                ),
            ]
        )
        question_chain = question_prompt | self.llm | StrOutputParser()
        question_queries = [
            {
                "user_request": user_request,
                "persona_name": persona.name,
                "persona_background": persona.background
            }
            for persona in personas
        ]
        return question_chain.batch(question_queries)
    
    def _generate_answers(self, personas: list[Persona], questions: list[str]) -> list[str]:
        answer_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたは以下のペルソナとして回答しています：{persona_name} - {persona_background}",
                ),
                ("human", "質問: {question}")
            ]
        )
        answer_chain = answer_prompt | self.llm | StrOutputParser()

        answer_queries = [
            {
                "persona_name": persona.name,
                "persona_background": persona.background,
                "question": question
            }
            for persona, question in zip(personas, questions)
        ]
        return answer_chain.batch(answer_queries)
    
    def _create_interviews(self, personas: list[Persona], questions: list[str], answers: list[str]) -> list[Interview]:
        return [
            Interview(
                persona=persona,
                question=question,
                answer=answer
            )
            for persona, question, answer in zip(personas, questions, answers)
        ]

In [36]:
class InformationEvaluator:
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm.with_structured_output(EvaluationResult)
    
    def run(self, user_request: str, interviews: list[Interview]) -> EvaluationResult:
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたは包括的な要件文書を作成するための情報の十分性を評価する専門家です。"
                ),
                (
                    "human",
                    f"以下のユーザーリクエストとインタビュー結果に基づいて、情報が十分かどうかを評価してください。\n\n"
                    "ユーザーリクエスト: {user_request}\n\n"
                    "インタビュー結果:\n{interview_results}\n\n"
                    "評価には、十分であるかどうかの判断と、その理由を含めてください。"
                ),
            ]
        )
        interview_summaries = "\n".join(
            [f"ペルソナ: {interview.persona.name}, 質問: {interview.question}, 回答: {interview.answer}" for interview in interviews]
        )
        chain = prompt | self.llm
        return chain.invoke(
            {
                "user_request": user_request,
                "interview_results": "\n".join(
                    f"ペルソナ: {i.persona.name} - {i.persona.background}\n"
                    "質問: {i.question}\n 回答: {i.answer}\n"
                    for i in interviews
                )
            }
        )

In [37]:
class RequirementsDocumentGenerator:
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm
    
    def run(self, user_request: str, interviews: list[Interview]) -> str:
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたは収集した情報に基づいて要件定義書を作成する専門家です。"
                ),
                (
                    "human",
                    f"以下のユーザーリクエストと複数のペルソナからのインタビュー結果に基づいて、要件定義書を作成してください。\n\n"
                    "ユーザーリクエスト: {user_request}\n\n"
                    "インタビュー結果:\n{interview_results}\n\n"
                    "要件定義書には以下のセクションを含めてください:\n"
                    "1. プロジェクト概要\n"
                    "2. 主要機能\n"
                    "3. 非機能要件\n"
                    "4. 制約条件\n"
                    "5. ターゲットユーザー\n"
                    "6. 優先順位\n"
                    "7. リスクと軽減策\n\n"
                    "出力は必ず日本語で行ってください。\n\n 要件文書:",

                ),
            ]
        )
        chain = prompt | self.llm | StrOutputParser()
        return chain.invoke(
            {
                "user_request": user_request,
                "interview_results": "\n".join(
                    f"ペルソナ: {i.persona.name} - {i.persona.background}\n"
                    "質問: {i.question}\n 回答: {i.answer}\n"
                    for i in interviews
                )
            }
        )

In [38]:
from langgraph.graph import StateGraph
from typing import Optional, Any
from langgraph.graph import END

class DocumentationAgent:
    def __init__(self, llm: ChatOpenAI, k: Optional[int] = None):
        self.persona_generator = PersonaGenerator(llm, k=k or 5)
        self.interview_conductor = InterviewConductor(llm)
        self.information_evaluator = InformationEvaluator(llm)
        self.requirements_generator = RequirementsDocumentGenerator(llm)

        self.graph = self._create_graph()

    def _create_graph(self) -> StateGraph:
        workflow = StateGraph(InterviewState)

        workflow.add_node("generate_personas", self._generate_personas)
        workflow.add_node("conduct_interviews", self._conduct_interviews)
        workflow.add_node("evaluate_information", self._evaluate_information)
        workflow.add_node("generate_requirements", self._generate_requirements)

        workflow.set_entry_point("generate_personas")

        workflow.add_edge("generate_personas", "conduct_interviews")
        workflow.add_edge("conduct_interviews", "evaluate_information")

        workflow.add_conditional_edges(
            "evaluate_information",
            lambda state: not state.is_information_sufficient and state.iteration < 5,
            {
                True: "generate_personas",
                False: "generate_requirements"
            },
        )
        workflow.add_edge("generate_requirements", END)

        return workflow.compile()
    
    def _generate_personas(self, state: InterviewState) -> dict[str, Any]:
        new_personas: Personas = self.persona_generator.run(state.user_request)
        return {
            "personas": new_personas.personas,
            "iteration": state.iteration + 1
        }
    
    def _conduct_interviews(self, state: InterviewState) -> dict[str, Any]:
        new_interviews: InterviewResult = self.interview_conductor.run(
            state.user_request, state.personas[-5:]
        )
        return {
            "interviews": new_interviews.interviews
        }

    def _evaluate_information(self, state: InterviewState) -> dict[str, Any]:
        evaluation: EvaluationResult = self.information_evaluator.run(
            state.user_request, state.interviews
        )
        return {
            "is_information_sufficient": evaluation.is_sufficient,
            "evaluateion_reason": evaluation.reason
        }
    
    def _generate_requirements(self, state: InterviewState) -> dict[str, Any]:
        requirements_doc: str = self.requirements_generator.run(
            state.user_request, state.interviews
        )
        return {
            "requirements_doc": requirements_doc
        }
    
    def run(self, user_request: str) -> str:
        initial_state = InterviewState(user_request=user_request)
        final_state = self.graph.invoke(initial_state)
        return final_state["requirements_doc"]

In [39]:
# 実行
llm = ChatOpenAI(model="gpt-4", temperature=0)
agent = DocumentationAgent(llm, k=5)
requirements_document = agent.run(user_request="スマートフォン向けの健康管理アプリを開発したい")
print(requirements_document)

1. プロジェクト概要:
本プロジェクトは、スマートフォン向けの健康管理アプリの開発を目指しています。このアプリは、ユーザーが自身の健康状態を管理し、改善するためのツールとなることを目指しています。

2. 主要機能:
- 食事記録: ユーザーが日々の食事を記録できる機能。
- 運動記録: ユーザーが日々の運動を記録できる機能。
- 健康状態のトラッキング: ユーザーが体重、血圧、睡眠時間などの健康指標を記録し、トラッキングできる機能。
- レコメンデーション: ユーザーの健康状態と目標に基づいた食事や運動の提案機能。

3. 非機能要件:
- ユーザーフレンドリー: 操作が直感的で、全ての年齢層のユーザーが利用できるようにする。
- レスポンシブデザイン: スマートフォンの異なる画面サイズに対応する。
- データセキュリティ: ユーザーの個人情報と健康データを安全に保管し、プライバシーを保護する。

4. 制約条件:
- データプライバシーとセキュリティ: 個人情報保護法や健康情報の取り扱いに関する法律を遵守する必要がある。
- デバイス互換性: iOSとAndroidの両方のプラットフォームで動作するようにする。

5. ターゲットユーザー:
- 健康意識の高い若者から中年層
- スポーツを愛する大学生
- 健康問題を抱える中高年層
- 忙しい生活を送る看護師やフリーランスのプロフェッショナル

6. 優先順位:
- 健康状態のトラッキング機能
- 食事と運動の記録機能
- レコメンデーション機能

7. リスクと軽減策:
- リスク: ユーザーのプライバシー侵害。軽減策: 厳格なデータセキュリティとプライバシーポリシーの実施。
- リスク: ユーザーインターフェースが複雑すぎて使いにくい。軽減策: ユーザビリティテストを行い、フィードバックに基づいて改善する。
- リスク: 法規制の遵守。軽減策: 法律専門家と協力して、全ての法規制を遵守するようにする。
